# Adversarial Attacks:

1. **Additive White Gaussian Noise (AWGN) Attack**: This attack involves adding carefully crafted noise to the input EEG signals to make them misclassified by the model. The added noise is usually imperceptible to humans but can significantly affect the model's performance.
   - White noise can be generated such that it has a mean of 0 and a specified standard deviation.
   - Noise level: You should experiment with different levels to understand at what point the model's performance starts to degrade.
   - Applying AWGN to EEG Signals


In [ ]:
import numpy as np

def awgn_attack(signal, noise_level):
    mean = 0
    std_dev = noise_level
    white_noise = np.random.normal(mean, std_dev, signal.shape)
    return signal + white_noise


2. **Fast Gradient Sign Method (FGSM)**: FGSM is a popular adversarial attack that involves perturbing the input EEG signals in the direction of the gradient of the loss function with respect to the input. This results in small, but adversarially meaningful, perturbations that can cause misclassification by the model.
   - Need access to the model's parameters and the ability to compute gradients, which is possible in frameworks like TensorFlow or PyTorch.
   - Applying the FGSM Perturbation: The perturbation is computed as the sign of the gradient multiplied by a small factor (ε, epsilon). This factor controls the magnitude of the perturbation.
   - Using PyTorch,


In [ ]:
import torch
import torch.nn.functional as F

def fgsm_attack(model, loss_fn, inputs, labels, epsilon):
    # Ensure model's parameters won't update
    inputs.requires_grad = True

    # Forward pass
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)

    # Zero all existing gradients
    model.zero_grad()

    # Backward pass to get gradients
    loss.backward()

    # Get the sign of the gradients
    data_grad = inputs.grad.data

    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_input = inputs + epsilon * data_grad.sign()

    # Adding clipping to maintain [0,1] range if necessary
    perturbed_input = torch.clamp(perturbed_input, 0, 1)

    return perturbed_input


3. **Projected Gradient Descent (PGD)**: PGD is an iterative version of FGSM where the perturbations are applied multiple times with small step sizes while ensuring that the perturbed signals remain within a specified epsilon-ball around the original signals. PGD tends to produce more effective adversarial examples compared to FGSM.
   - PGD is like FGSM but applies the perturbation in multiple small steps. This iterative approach allows PGD to explore more of the input space around the original data.
   - Algorithm:
     - Start with a randomly perturbed version of the original input within the ε-ball
     - In each iteration, apply a small FGSM-like step, and then clip the result to ensure it stays within the ε-ball.
     - Repeat for a fixed number of iterations or until convergence.
   - Using PyTorch,


In [ ]:
import torch

def pgd_attack(model, inputs, labels, epsilon, alpha, num_iter):
    perturbed_inputs = inputs.clone()
    perturbed_inputs = perturbed_inputs + torch.rand_like(inputs) * 2 * epsilon - epsilon

    for _ in range(num_iter):
        perturbed_inputs.requires_grad = True
        outputs = model(perturbed_inputs)
        model.zero_grad()
        loss = F.cross_entropy(outputs, labels)
        loss.backward()

        with torch.no_grad():
            # FGSM step
            step = alpha * perturbed_inputs.grad.sign()
            perturbed_inputs = perturbed_inputs + step

            # Clipping step (project back to ε-ball)
            perturbed_inputs = torch.max(torch.min(perturbed_inputs, inputs + epsilon), inputs - epsilon)

    return perturbed_inputs


4. **Jacobian-based Saliency Map Attack (JSMA)**: JSMA identifies the most influential features (or electrodes in the case of EEG signals) and perturbs them to maximize the misclassification probability. It's based on computing the Jacobian matrix of the model's output with respect to the input features.
   - The attack is based on the concept of a saliency map, which is derived from the Jacobian matrix of the model's output with respect to the input.
   - Creating a saliency map:
     1. Using the Jacobian matrix, a saliency map is created that highlights which features should be perturbed to most effectively change the model's output.
     2. The goal is to find the features whose modification will either maximize the probability of a specific (incorrect) class or minimize the probability of the correct class.
   - Implementation involves computing the Jacobian matrix and then iteratively modifying the most influential features according to the saliency map.
   - Using PyTorch,


In [ ]:
# Simplified conceptual Python code
def jsma_attack(model, input_features, target_label, max_distortion):
    for i in range(max_distortion):
        jacobian = compute_jacobian(model, input_features)
        saliency_map = calculate_saliency_map(jacobian, target_label)
        most_influential_feature = identify_most_influential_feature(saliency_map)
        input_features = modify_feature(input_features, most_influential_feature)
        if model.predict(input_features) == target_label:
            break
    return input_features


5. **DeepFool Attack**: DeepFool is an iterative attack that aims to find the minimum perturbation required to cause misclassification by iteratively linearizing the decision boundary of the model and moving the input signals towards the decision boundary.
   - The key idea of DeepFool is to iteratively push the input data towards the model's decision boundary until it crosses into a different classification region, causing misclassification.
   - It tries to calculate the minimal perturbation required to change the classification, which gives an estimate of the model's robustness.
   - Iterative Linearization:
     - At each iteration, DeepFool approximates the model's decision boundary near the current data point as linear.
     - It then calculates the minimal perturbation needed to reach this linearized decision boundary.
   - Moving Towards the Decision Boundary:
     - The process is repeated iteratively until the altered input crosses the boundary, resulting in a different classification.
   - Using PyTorch,


In [ ]:
# Pseudocode for DeepFool
def deepfool_attack(model, input_data, num_classes):
    perturbed_data = input_data.clone()
    for _ in range(max_iterations):
        gradients = compute_gradients(model, perturbed_data, num_classes)
        perturbation = calculate_minimal_perturbation(gradients)
        perturbed_data += perturbation
        if model.predict(perturbed_data) != model.predict(input_data):
            break
    return perturbed_data


6. **CleverHans Library**: CleverHans is a Python library for benchmarking adversarial attacks against machine learning models. It provides implementations of various adversarial attacks, including FGSM, PGD, and JSMA, making it a useful tool for experimenting with different attack strategies on EEG classification models.
   - Install cleverhans, `pip install cleverhans`
   - CleverHans offers a range of functions to implement various adversarial attacks
   - Choose the attack you want to apply, such as FGSM, PGD, or JSMA. Configure the parameters of the attack according to your testing needs.
   - Using cleverhans for attacks:
     - For example, to use FGSM, you would set up the attack with the appropriate parameters like epsilon (the attack strength).
     - Generate the adversarial examples using your EEG data as inputs.


In [ ]:
!pip install cleverhans

In [ ]:
from cleverhans.future.tf2.attacks import fast_gradient_method

# Assuming `model` is your EEG classifier and `x_test` is the input data
fgsm_params = {'epsilon': 0.3, 'clip_min': 0., 'clip_max': 1.}
adv_x = fast_gradient_method(model, x_test, **fgsm_params)


# Robustness:

- **Noise Addition**:
  - White Noise: Random noise that has equal intensity at different frequencies, providing a constant power spectral density.

You can generate white noise programmatically using random functions in most data analysis software.


In [ ]:
import numpy as np

# Example: Adding white noise to EEG data
def add_white_noise(eeg_data, noise_level=0.1):
    # Generate white noise
    white_noise = np.random.normal(0, noise_level, eeg_data.shape)
    # Add noise to the EEG data
    noisy_eeg = eeg_data + white_noise
    return noisy_eeg


- **Data Perturbation Techniques**:
  - Time Warping: Slightly alter the time axis of the EEG signals to simulate variations in signal speed.
    - Methods: Expand or reduce the time axis
    - Python: numpy or scipy
  - Warping factors: A warping factor > 1 stretches the signal (slows it down), and < 1 compresses it (speeds it up)


In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

def time_warp(signal, warping_factor):
    n = len(signal)
    original_time = np.linspace(0, 1, n)
    warped_time = np.linspace(0, 1, int(warping_factor * n))

    interpolator = interp1d(original_time, signal, kind='linear')
    warped_signal = interpolator(warped_time)

    return warped_signal

# Example usage
eeg_signal = np.sin(np.linspace(0, 10*np.pi, 1000))  # A sample EEG-like signal
warped_signal = time_warp(eeg_signal, 0.8)  # Warping factor < 1 compresses the signal

plt.plot(eeg_signal, label='Original Signal')
plt.plot(warped_signal, label='Warped Signal')
plt.legend()
plt.show()
